# Alpha Vantage

In [135]:
# Load libs
import csv
import requests
import time

# Set params for API
api_key = 'SWJ8LKHNRYM6AP15'
ticker = 'SPY'
interval_mins = 60
times = [f'year{x}month{y}' for x in range(1, 3) for y in range(1, 13)]

# Get url for each "slice"
urls = []
for time_slice in times:
    urls.append(f'''https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={ticker}&interval={interval_mins}min&apikey={api_key}&outputsize=full&slice={time_slice}''')

In [136]:
columns = ['time', 'open', 'high', 'low', 'close', 'volume']
stock_data_df = pd.DataFrame(columns = columns)

for url in urls[:]:
    # Get data from each url in list
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode('utf-8')
        cr = list(csv.reader(decoded_content.splitlines(), delimiter=','))
    # Get data into dataframe
    df = pd.DataFrame(cr[1:], columns = columns)
    df[['open', 'high', 'low', 'close']] = df[['open', 
                                               'high', 
                                               'low', 
                                               'close']].apply(pd.to_numeric).round(2)
    df['volume'] = df['volume'].apply(pd.to_numeric)
    df['time'] = df['time'].apply(pd.to_datetime)
    stock_data_df = pd.concat([stock_data_df, df])
    # Wait for API calls to refresh
    time.sleep(15)

In [162]:
# View data
stock_data_df.info()
stock_data_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7912 entries, 0 to 328
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    7912 non-null   datetime64[ns]
 1   open    7912 non-null   float64       
 2   high    7912 non-null   float64       
 3   low     7912 non-null   float64       
 4   close   7912 non-null   float64       
 5   volume  7912 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 432.7+ KB


,time,open,high,low,close,volume
0,2022-11-07 20:00:00,380.03,381.11,380.01,380.83,118848
1,2022-11-07 19:00:00,379.71,380.04,379.38,379.95,79734
2,2022-11-07 18:00:00,379.61,379.82,379.45,379.71,67735
3,2022-11-07 17:00:00,379.94,380.04,379.44,379.68,8845748
4,2022-11-07 16:00:00,379.34,380.57,379.32,379.94,14645881


In [166]:
# Write data to .csv
# stock_data_df.to_csv("Hourly_SPY_Data.csv", sep=',', index=False, mode = 'w')
# Write to .pkl
# stock_data_df.to_pickle("Hourly_SPY_Data.pkl")

# Scrape Prices

In [27]:
import requests
from bs4 import BeautifulSoup
import json

def getData(stock):
    # Set user agent
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
    
    # URL to scrape from
    url = f'https://finance.yahoo.com/quote/{stock}'
    
    # Get soup
    r = requests.get(url, headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Get data from HTML
    data = {
        'symbol': stock,
        'price': float(soup.find('fin-streamer', {"class": "Fw(b) Fz(36px) Mb(-4px) D(ib)"}).text),
        'change': float(soup.find('fin-streamer', {"class": "Fw(500) Pstart(8px) Fz(24px)"}).text)
    }
    return data

In [156]:
# List of stocks to get data on
stocks = ['TSLA', 'MSFT', 'AAPL', 'GOOGL']

# Get data for each stocks
stock_data = []
for idx, tick in enumerate(stocks):
    stock_data.append(getData(tick))
    print(f"{tick}: ${stock_data[idx].get('price')}")

TSLA: $191.3
MSFT: $228.87
AAPL: $139.5
GOOGL: $88.9
